In [ ]:
import pandas as pd
data = pd.read_csv('/content/NER dataset.csv', encoding= 'unicode_escape')
data.head()

In [ ]:
from itertools import chain
def get_dict_map(data, token_or_tag):
    tok2idx = {}
    idx2tok = {}

    if token_or_tag == 'token':
        vocab = list(set(data['Word'].to_list()))
    else:
        vocab = list(set(data['Tag'].to_list()))

    idx2tok = {idx:tok for  idx, tok in enumerate(vocab)}
    tok2idx = {tok:idx for  idx, tok in enumerate(vocab)}
    return tok2idx, idx2tok
token2idx, idx2token = get_dict_map(data, 'token')
tag2idx, idx2tag = get_dict_map(data, 'tag')

In [ ]:
data['Word_idx'] = data['Word'].map(token2idx)
data['Tag_idx'] = data['Tag'].map(tag2idx)
data_fillna = data.ffill()
# Groupby and collect columns
data_group = data_fillna.groupby(
['Sentence #'],as_index=False
)[['Word', 'POS', 'Tag', 'Word_idx', 'Tag_idx']].agg(lambda x: list(x))

In [ ]:
from sklearn.model_selection import train_test_split
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

def get_pad_train_test_val(data_group, data):

    #get max token and tag length
    n_token = len(list(set(data['Word'].to_list())))
    n_tag = len(list(set(data['Tag'].to_list())))

    #Pad tokens (X var)
    tokens = data_group['Word_idx'].tolist()
    maxlen = max([len(s) for s in tokens])
    pad_tokens = pad_sequences(tokens, maxlen=maxlen, dtype='int32', padding='post', value= n_token - 1)

    #Pad Tags (y var) and convert it into one hot encoding
    tags = data_group['Tag_idx'].tolist()
    pad_tags = pad_sequences(tags, maxlen=maxlen, dtype='int32', padding='post', value= tag2idx["O"])
    n_tags = len(tag2idx)
    pad_tags = [to_categorical(i, num_classes=n_tags) for i in pad_tags]

    #Split train, test and validation set
    tokens_, test_tokens, tags_, test_tags = train_test_split(pad_tokens, pad_tags, test_size=0.1, train_size=0.9, random_state=2020)
    train_tokens, val_tokens, train_tags, val_tags = train_test_split(tokens_,tags_,test_size = 0.25,train_size =0.75, random_state=2020)

    print(
        'train_tokens length:', len(train_tokens),
        '\ntrain_tokens length:', len(train_tokens),
        '\ntest_tokens length:', len(test_tokens),
        '\ntest_tags:', len(test_tags),
        '\nval_tokens:', len(val_tokens),
        '\nval_tags:', len(val_tags),
    )

    return train_tokens, val_tokens, test_tokens, train_tags, val_tags, test_tags

train_tokens, val_tokens, test_tokens, train_tags, val_tags, test_tags = get_pad_train_test_val(data_group, data)

train_tokens length: 32372 
train_tokens length: 32372 
test_tokens length: 4796 
test_tags: 4796 
val_tokens: 10791 
val_tags: 10791


In [ ]:
import numpy as np
import tensorflow
from tensorflow.keras import Sequential, Model, Input
from tensorflow.keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional
from tensorflow.keras.utils import plot_model
from numpy.random import seed
seed(1)
tensorflow.random.set_seed(2)

In [ ]:
input_dim = len(list(set(data['Word'].to_list())))+1
output_dim = 64
input_length = max([len(s) for s in data_group['Word_idx'].tolist()])
n_tags = len(tag2idx)

In [ ]:
def get_bilstm_lstm_model():
    model = Sequential()

    # Add Embedding layer
    model.add(Embedding(input_dim=input_dim, output_dim=output_dim, input_length=input_length))

    # Add bidirectional LSTM
    model.add(Bidirectional(LSTM(units=output_dim, return_sequences=True, dropout=0.2, recurrent_dropout=0.2), merge_mode = 'concat'))

    # Add LSTM
    model.add(LSTM(units=output_dim, return_sequences=True, dropout=0.5, recurrent_dropout=0.5))

    # Add timeDistributed Layer
    model.add(TimeDistributed(Dense(n_tags, activation="relu")))

    #Optimiser
    # adam = k.optimizers.Adam(lr=0.0005, beta_1=0.9, beta_2=0.999)

    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.summary()

    return model

In [ ]:
def train_model(X, y, model):
    loss = list()
    for i in range(25):
        # fit model for one epoch on this sequence
        hist = model.fit(X, y, batch_size=1000, verbose=1, epochs=1, validation_split=0.2)
        loss.append(hist.history['loss'][0])
    return loss

In [ ]:
results = pd.DataFrame()
model_bilstm_lstm = get_bilstm_lstm_model()
model_bilstm_lstm.build(input_shape=(None, input_length))
plot_model(model_bilstm_lstm)
results['with_add_lstm'] = train_model(train_tokens, np.array(train_tags), model_bilstm_lstm)

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_1 (Bidirectional) │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_3 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_1              │ ?                      │   0 (unbuilt) │
│ (TimeDistributed)               │                        │               │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

26/26 ━━━━━━━━━━━━━━━━━━━━ 208s 7s/step - accuracy: 0.7934 - loss: 2.1909 - val_accuracy: 0.9681 - val_loss: 0.4315
26/26 ━━━━━━━━━━━━━━━━━━━━ 175s 7s/step - accuracy: 0.9676 - loss: 0.4245 - val_accuracy: 0.9681 - val_loss: 0.3329
26/26 ━━━━━━━━━━━━━━━━━━━━ 191s 7s/step - accuracy: 0.9676 - loss: 0.3634 - val_accuracy: 0.9681 - val_loss: 0.3066
26/26 ━━━━━━━━━━━━━━━━━━━━ 179s 7s/step - accuracy: 0.9675 - loss: 0.3312 - val_accuracy: 0.9681 - val_loss: 0.2949
26/26 ━━━━━━━━━━━━━━━━━━━━ 164s 6s/step - accuracy: 0.9676 - loss: 0.3149 - val_accuracy: 0.9681 - val_loss: 0.2706
26/26 ━━━━━━━━━━━━━━━━━━━━ 176s 7s/step - accuracy: 0.9676 - loss: 0.3081 - val_accuracy: 0.9682 - val_loss: 0.3105
26/26 ━━━━━━━━━━━━━━━━━━━━ 187s 7s/step - accuracy: 0.9675 - loss: 0.3372 - val_accuracy: 0.9682 - val_loss: 0.2973
26/26 ━━━━━━━━━━━━━━━━━━━━ 176s 7s/step - accuracy: 0.9675 - loss: 0.3308 - val_accuracy: 0.9681 - val_loss: 0.2730
26/26 ━━━━━━━━━━━━━━━━━━━━ 176s 7s/step - accuracy: 0.9676 - loss: 0.294

In [ ]:
import spacy
from spacy import displacy
nlp = spacy.load('en_core_web_sm')
text = nlp('Hi, My name is Divyasri \n I am from India \n I want to work with Google \n Steve Jobs is My Inspiration')
displacy.render(text, style = 'ent', jupyter=True)

In [ ]:
import pickle

# Save the trained model
with open('ner_model.pkl', 'wb') as f:
    pickle.dump(model_bilstm_lstm, f)

# Save token and tag mappings
with open('token2idx.pkl', 'wb') as f:
    pickle.dump(token2idx, f)

with open('idx2token.pkl', 'wb') as f:
    pickle.dump(idx2token, f)

with open('tag2idx.pkl', 'wb') as f:
    pickle.dump(tag2idx, f)

with open('idx2tag.pkl', 'wb') as f:
    pickle.dump(idx2tag, f)


In [ ]:
# --- SAVE PART (run once in Colab) ---

# 1) Save the model in Keras format (NOT pickle)
model_bilstm_lstm.save("ner_model.keras")

# 2) Save only the dictionaries with pickle
import pickle

with open("token2idx.pkl", "wb") as f:
    pickle.dump(token2idx, f)

with open("idx2tag.pkl", "wb") as f:
    pickle.dump(idx2tag, f)
